<a href="https://colab.research.google.com/github/JPauloFavarettoSilva/RedesGenerativasGANs/blob/main/RedesGANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential

# Função para construir o gerador
def build_generator():
    model = Sequential([  # Cria um modelo sequencial
        layers.Dense(16, activation='relu', input_dim=1),  # Camada densa com 16 neurônios e ativação ReLU, esperando uma entrada de dimensão 1
        layers.Dense(1)  # Camada densa com 1 neurônio, sem ativação
    ])
    return model  # Retorna o modelo gerador

# Função para construir o discriminador
def build_discriminator():
    model = Sequential([  # Cria um modelo sequencial
        layers.Dense(16, activation='relu', input_dim=1),  # Camada densa com 16 neurônios e ativação ReLU, esperando uma entrada de dimensão 1
        layers.Dense(1, activation='sigmoid')  # Camada densa com 1 neurônio e ativação sigmoide para produzir uma probabilidade
    ])
    return model  # Retorna o modelo discriminador

# Construindo o gerador e o discriminador
generator = build_generator()
discriminator = build_discriminator()

# Compila o discriminador com o otimizador Adam e a função de perda de entropia cruzada binária
discriminator.compile(optimizer='adam', loss='binary_crossentropy')

# Conectando o gerador ao discriminador
discriminator.trainable = False  # O discriminador não será treinado enquanto o gerador estiver sendo treinado
gan = Sequential([generator, discriminator])  # GAN é um modelo sequencial que consiste em gerador seguido pelo discriminador
gan.compile(optimizer='adam', loss='binary_crossentropy')  # Compila o GAN com o otimizador Adam e a função de perda de entropia cruzada binária

# Dados reais e rótulos
real_data = np.array([[6.0], [7.0], [8.0]])  # Dados reais
labels_real = np.ones((3, 1))  # Rótulos para dados reais (1)
labels_fake = np.zeros((3, 1)) # Rótulos para dados falsos (0)

epochs = 10  # Número de épocas para treinamento
batch_size = 3  # Tamanho do lote

# Loop de treinamento
for epoch in range(epochs):
    # Gerando dados falsos (números)
    noise = np.random.normal(0, 1, (batch_size, 1))  # Ruído aleatório
    generated_data = generator.predict(noise)  # Gerando dados usando o gerador

    # Treinando o discriminador com dados reais e falsos
    d_loss_real = discriminator.train_on_batch(real_data, labels_real)  # Treinando o discriminador com dados reais
    d_loss_fake = discriminator.train_on_batch(generated_data, labels_fake)  # Treinando o discriminador com dados falsos

    # Treinando o gerador (enganar o discriminador)
    noise = np.random.normal(0, 1, (batch_size, 1))  # Novo ruído aleatório
    gan_loss = gan.train_on_batch(noise, labels_real)  # Treinando o GAN para enganar o discriminador

    # Exibindo informações de perda a cada 100 épocas
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss D Real: {d_loss_real}, Loss D Fake: {d_loss_fake}, Loss G: {gan_loss}")

# Gerando números usando o gerador treinado
noise = np.random.normal(0, 1, (5, 1))  # Novo ruído aleatório
generated_numbers = generator.predict(noise)  # Gerando números
print("Números gerados:", generated_numbers)  # Imprimindo os números gerados


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Epoch 0, Loss D Real: 1.224561333656311, Loss D Fake: 0.9511186480522156, Loss G: [array(0.95111865, dtype=float32), array(0.95111865, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Números gerados: [[-0.00473134]
 [ 0.27767694]
 [ 0.03426722]
 [ 0.23820573]
 [ 0.1513302 ]]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.datasets import mnist

# Carregar o conjunto de dados MNIST
(x_train, _), (_, _) = mnist.load_data()  # Carregando apenas as imagens de treino (ignorando os rótulos)
x_train = (x_train - 127.5) / 127.5  # Normalizar os dados para o intervalo [-1, 1]
x_train = np.expand_dims(x_train, axis=-1)  # Adicionar a dimensão do canal (necessário para a rede)

# Parâmetros
batch_size = 64  # Tamanho do lote para treinamento
epochs = 100  # Número de épocas
latent_dim = 100  # Dimensão do espaço latente (tamanho do ruído de entrada para o gerador)

# Construir o Gerador
def build_generator():
    model = Sequential([  # Usando um modelo sequencial
        layers.Dense(256, activation='relu', input_dim=latent_dim),  # Primeira camada densa com 256 neurônios e ativação ReLU
        layers.BatchNormalization(),  # Normalização em lote para estabilizar o treinamento
        layers.Dense(512, activation='relu'),  # Segunda camada densa com 512 neurônios
        layers.BatchNormalization(),
        layers.Dense(1024, activation='relu'),  # Terceira camada densa com 1024 neurônios
        layers.BatchNormalization(),
        layers.Dense(28 * 28 * 1, activation='tanh'),  # Camada de saída que gera uma imagem 28x28 com 1 canal (escala de cinza)
        layers.Reshape((28, 28, 1))  # Reshape para dar à saída a forma correta de imagem 28x28x1
    ])
    return model  # Retorna o modelo gerador

# Construir o Discriminador
def build_discriminator():
    model = Sequential([  # Usando um modelo sequencial
        layers.Flatten(input_shape=(28, 28, 1)),  # Achata a entrada 28x28x1 em um vetor 1D
        layers.Dense(512, activation='relu'),  # Primeira camada densa com 512 neurônios
        layers.Dense(256, activation='relu'),  # Segunda camada densa com 256 neurônios
        layers.Dense(1, activation='sigmoid')  # Camada de saída que produz uma probabilidade (1 neurônio com ativação sigmoide)
    ])
    return model  # Retorna o modelo discriminador

# Construir o GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Durante o treinamento do GAN, o discriminador não será atualizado
    model = Sequential([generator, discriminator])  # O GAN consiste no gerador seguido pelo discriminador
    return model  # Retorna o modelo GAN

# Instanciar o Gerador e o Discriminador
generator = build_generator()  # Cria o gerador
discriminator = build_discriminator()  # Cria o discriminador
discriminator.compile(optimizer='adam', loss='binary_crossentropy')  # Compila o discriminador com função de perda binária

# Instanciar o GAN
gan = build_gan(generator, discriminator)  # Cria o modelo GAN
gan.compile(optimizer='adam', loss='binary_crossentropy')  # Compila o GAN com a função de perda binária

# Treinar a GAN
for epoch in range(epochs):
    # Treinar o Discriminador
    idx = np.random.randint(0, x_train.shape[0], batch_size)  # Seleciona amostras reais aleatórias do conjunto de dados
    real_images = x_train[idx]  # Extrai as imagens reais
    real_labels = np.ones((batch_size, 1))  # Rótulos para as imagens reais (1)

    noise = np.random.normal(0, 1, (batch_size, latent_dim))  # Gera ruído aleatório para o gerador
    fake_images = generator.predict(noise)  # Gera imagens falsas usando o gerador
    fake_labels = np.zeros((batch_size, 1))  # Rótulos para as imagens falsas (0)

    # Treinando o discriminador com dados reais e falsos
    d_loss_real = discriminator.train_on_batch(real_images, real_labels)  # Treina com as imagens reais
    d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)  # Treina com as imagens falsas

    # Treinar o Gerador (através do GAN)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))  # Gera novo ruído aleatório
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))  # Treina o gerador para enganar o discriminador (rótulos = 1)

    # A cada 1000 épocas, exibir a perda e gerar imagens
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, D Loss Real: {d_loss_real}, D Loss Fake: {d_loss_fake}, G Loss: {g_loss}")

        # Gerar e visualizar imagens
        noise = np.random.normal(0, 1, (16, latent_dim))  # Gera ruído para criar 16 imagens
        generated_images = generator.predict(noise)  # Gera imagens a partir do ruído
        generated_images = (generated_images + 1) / 2.0  # Reverter a normalização para o intervalo [0, 1]

        # Plotar as imagens geradas
        fig, axs = plt.subplots(4, 4)  # Cria uma grade 4x4 para exibir as imagens
        cnt = 0
        for i in range(4):
            for j in range(4):
                axs[i, j].imshow(generated_images[cnt, :, :, 0], cmap='gray')  # Exibe a imagem gerada em escala de cinza
                axs[i, j].axis('off')  # Remove os eixos
                cnt += 1
        plt.show()  # Exibe o gráfico
